In [2]:
import os
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from IPython.display import clear_output
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [4]:
def _loader(path: str) -> Image.Image:
    with open(path, "rb") as f:
        img = Image.open(f)
        return img.conver("L")

In [5]:
images = glob(os.path.join("faces_dataset/train", "*/*"))

In [14]:
class Faces(Dataset):
    def __init__(self, root: str, split: str = "train", transform=None):
        self.root = os.path.join(root, split)
        self.names, self.name2index, self.index2name = self._get_names()
        self.images, self.labels = self._get_people()
        self.total_idxs = np.arange(len(self.images))
        self.transform = transform
        
    def _get_names(self):
        names = list(map(lambda x: int(x), os.listdir(self.root)))
        
        name2index = {name: i for i, name in enumerate(names)}
        index2name = {i: name for i, name in enumerate(names)}
        
        return names, name2index, index2name
    
    def _get_people(self):
        images = glob(os.path.join(self.root, "*/*"))
        labels = [self.name2index[int(image.split("/")[-2])] for image in images]
        return np.array(images), np.array(labels)
    
    def __getitem__(self, item):
        image1 = _loader(self.images[item])
        
        if np.random.uniform() < 0.5:
            image2 = _loader(np.random.choice(self.images[self.labels == self.labels[item]]))
            label = torch.tensor([1], dtype=torch.int)
        else:
            image2 = _loader(np.random.choice(self.images[self.labels != self.labels[item]]))
            label = torch.tensor([0], dtype=torch.int)
        
        return image1, image2, label, torch.tensor([self.labels[item]], dtype=torch.int)
    
    def __len__(self):
        return len(self.images)

In [15]:
dataset = Faces('faces_dataset')

IndexError: list index out of range